In [ ]:
!pip install bs4 requests pandas

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
multi_page_result = list()

for i in range(0, 15):
  url = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page=" + str(i)
  headers = {'User-Agent':'Chrome/66.0.3359.181'}
  html = requests.get(url,headers=headers) ##requests 를 이용해서 url의 html 파일을 가져옴
  soup = BeautifulSoup(html.text, "html.parser") ##가져온 html 파일을 html parser를 통해서 정리

  platform_list = list()
  html_platform_list = soup.findAll("div", attrs={"class":"platform"})
  first = 0
  for line in html_platform_list:
    text = line.get_text()
    platform_list.append(text)
  
  preprocess_platform = list()
  for platform_text in platform_list:
    platform_text = platform_text.replace("\n", "")
    platform_text = platform_text.replace("Platform:", "")
    platform_text = platform_text.replace("All Platforms","")
    platform_text = platform_text.strip()
    platform_text = platform_text.replace(" ", "-").strip() #공백을 -로 변환
    platform_text = platform_text.lower()
    if(first == 0):
      first = first+1
    else:
      preprocess_platform.append(platform_text)

  detail = list()
  detail = soup.findAll("a", attrs={"class":"title"})
  name = list()
  for i in range(0,100): #게임 개수만큼
    for line in detail[i]:
     text = line.get_text()
     text = text.replace(" &", "")
     text = text.replace(":", "")          # : 를 없앰
     text = text.replace(";", "")
     text = text.lower()
     text = text.replace("'", "")
     text = text.replace(".", "")
     text = text.replace(",", "")
     text = text.replace("#", "")
     text = text.replace("$!", "!")
     text = text.replace(" /", "")
     text = text.replace(" ", "-").strip() #공백을 -로 변환
     name.append(text)
  
  preprocess_date = list() 
  preprocess_genre = list()
  preprocess_userscore = list()
  preprocess_metascore = list()
  new_genre = list()  
  
  for j in range(0,100): 
    url2_1 = "https://www.metacritic.com/game/"
    url2_2 = preprocess_platform[j]+"/"
    url2_3 = name[j]
    url2 = url2_1 + url2_2 + url2_3
  
    headers2 = {'User-Agent':'Chrome/66.0.3359.181'}
    html2 = requests.get(url2,headers=headers2)
    soup2 = BeautifulSoup(html2.text, 'html.parser')

    date_detail = list()
    date = list()
    genre_detail = list()
    genre = list()

    date_detail = soup2.findAll("li", attrs={"class":"summary_detail release_data"})
    print(date_detail)
    print(name[j])
    date = date_detail[0].find_all('span',{'class':'data'})
    for art in date:
      art = art.text.strip()
      preprocess_date.append(art)

    usescore = list()
    pu_score = list()
    mu_score = list()
    nu_score = list()
    pm_score = list()
    mm_score = list()
    nm_score = list()
    usescore = soup2.findAll("a", attrs={"class":"metascore_anchor"})
    pu_score = usescore[1].findAll("div", attrs={"class":"metascore_w user large game positive"})
    mu_score = usescore[1].findAll("div", attrs={"class":"metascore_w user large game mixed"})
    nu_score = usescore[1].findAll("div", attrs={"class":"metascore_w user large game negative"})
    tu_score = usescore[1].findAll("div", attrs={"class":"metascore_w user large game tbd"})
    pm_score = usescore[0].findAll("div", attrs={"class":"metascore_w xlarge game positive"})
    mm_score = usescore[0].findAll("div", attrs={"class":"metascore_w xlarge game mixed"})
    nm_score = usescore[0].findAll("div", attrs={"class":"metascore_w xlarge game negative"})
    for art in pu_score:
      art = art.text.strip()
      preprocess_userscore.append(art)
    for art in mu_score:
      art = art.text.strip()
      preprocess_userscore.append(art)
    for art in nu_score:
      art = art.text.strip()
      preprocess_userscore.append(art)
    for art in tu_score:
      art = art.text.strip()
      preprocess_userscore.append(art)
    for art in pm_score:
      art = art.text.strip()
      preprocess_metascore.append(art)
    for art in mm_score:
      art = art.text.strip()
      preprocess_metascore.append(art)
    for art in nm_score:
      art = art.text.strip()
      preprocess_metascore.append(art)

    html_label_list = soup.findAll("div",attrs = {"metascore_w"})
    label_list = list()

  for line in html_label_list:
    if(line.attrs["class"][1] == 'user'):
      label = line.attrs["class"][4]
    else:
      label = line.attrs["class"][3]
    label_list.append(label)
  
  result_list = list()
  for label, platform, name, date, metascore, userscore in zip(label_list, preprocess_platform,
                                                               name, preprocess_date, 
                                                               preprocess_metascore, preprocess_userscore):
    row_data = [label, platform, name, date, metascore, userscore]
    multi_page_result.append(row_data)

In [ ]:
import pandas as pd

review_df = pd.DataFrame(multi_page_result, columns =['label', 'platform', 'name','date', 'metascore', 'userscore'])
review_df


In [ ]:
review_df.to_excel('review10~15.xlsx')